In [1]:
import json
import pandas as pd

league_id = '008da363-45af-4a07-87d2-f22567e27213'

from get_summoner_ids_from_league_id import get_summoner_ids_from_league_id
from get_puuid_by_summon_id import get_puuid_by_summon_id
from get_match_ids_by_puuid import get_match_ids_by_puuid
from get_match_timeline import get_match_timeline
from extract_events_from_timeline import extract_events_from_timeline

# Create rate limiter and get initial PUUids list

In [8]:
import time

class RateLimiter:
    def __init__(self, max_requests, period):
        self.max_requests = max_requests
        self.period = period
        self.timestamps = []

    def __call__(self):
        now = time.time()
        self.timestamps = [ts for ts in self.timestamps if ts > now - self.period]

        if len(self.timestamps) < self.max_requests:
            self.timestamps.append(now)
        else:
            oldest_request = min(self.timestamps)
            time_to_wait = self.period - (now - oldest_request)
            time.sleep(time_to_wait)
            self.timestamps.append(time.time())

rl_sec = RateLimiter(20, 1)  # 20 requests per 1 second
rl_min = RateLimiter(100, 120)  # 100 requests per 2 minutes

puuids = []
while len(puuids) < 20: 
    rl_sec()  # Apply rate limiting
    rl_min()  # Apply rate limiting

    summoner_ids = get_summoner_ids_from_league_id(league_id)
    
    for summoner_id in summoner_ids:
        rl_sec()  # Apply rate limiting
        rl_min()  # Apply rate limiting
        
        puuid = get_puuid_by_summon_id(summoner_id)
        puuids.append(puuid)

        if len(puuids) >= 20:
            break
    
print(f"Total puuids: {len(puuids)}")
print(puuids)


pd.DataFrame(puuids).to_csv('puuids.csv')

Total puuids: 20
['rOra4jm75Nj0y4-_7D7MVRfiBlpBXhpSonyI0rwwFacc0uV9fg2HC7PgC_nNJZ1WCXFL_EfTBrDLwQ', 'Cw5DTp0JHNYPKCvhFtw17wdGaFimvHUUckg4Fx8w2ZvYK7v9_mZtKHDOpInEGjGO5-5FZzFna-dDHg', 'ox60Cqe2-_wa_jLEVoy241kruFzWrD12QJd-RkxeBP5Mt3Kx5GA54dhPgfDTVtYnH7XbneQFgNi8aQ', 'RHZ59BUcE3vQxkwW_6x1sDkSXAjD9iR50Exf1nWDYYOzJdtpolEijub4KBwiqxamQYxUCfsRz2Ub9w', 'wwCb2LqGsnZUzOMCfC-YdN4tLb2SuPOwhw6yWPezsU9vG7FKoMGkazxBeWG_ebFYsynU6uQ70fqBuw', 'lnMmSlO27sHd6L0HgrYS5ivq8XMQDocSZytgMXsqMeu0dg3BiGxzfH7DGxAcBpuFpUDFIoDylKtvCg', 'rM1y8szWrtdn5qzlF-EA6S1jGpsZ65OC_4_6eaqmP26I5C8y2G-Z8wyqAc3G6RLCfJ9hTBBkSGMLFA', 'wylRJstyPPKMByoq_c4X59sItxo_Qhje0gXRZfQyl__YAa4O8OsjRhWpaNWTJ_byPdoLhalGzyyf3A', 'aFC0kPZY3mGhs0c-MT5blLeSuKiGzr9E9HJI9xEETp6piIgvNdYzx_xWo8f7emBqSmP_C6chMaVrvg', 'QH2QxpwEEEd0_BMQq1iydfZkVpVhDtCwe7QYeyNwd1vv5BWKJmlv2oYQuo_6phOOBKwB6U4ir6f2zA', '4aoPz1JnVaW0nY9N3rC-TwPR_5OORYX56Gpn2HDwe07bxrnT8yBIa1pof8FgL-eZ8aYGA9hYc8S8nQ', 'vRz9RPnMQ7uwkTVnNBZEF0F2ghwP9T_Xs_3hxKBdVrb4w0RSk3BTmbkGd4K866TivqC0G_9aVKJhwg'

# Get matchIds from PUUids

In [17]:
import random
import pandas as pd

# Create dict to hold match timelines
match_tl_dict = {}

# Create list to hold selected match IDs
selected_match_ids_list = []

# Iterate over PUUIDs
for puuid in puuids:
    # Apply rate limiting
    rl_sec()
    rl_min()

    # Get match IDs for each PUUID
    match_ids = get_match_ids_by_puuid(puuid)
    
    # Randomly select 10 (or up to 10) matches
    selected_match_ids = random.sample(match_ids, min(10, len(match_ids)))

    # Add selected match IDs to the list
    selected_match_ids_list.extend(selected_match_ids)

# Convert the list of selected match IDs to a DataFrame and save as CSV
pd.DataFrame(selected_match_ids_list, columns=['match_id']).to_csv('selected_match_ids.csv')
print('match_ids complete')

match_ids complete


# Get matchtimelines and save as JSON

In [19]:
import json
import os

# Create the 'match_timeline' folder if it doesn't exist
if not os.path.exists('match_timelines'):
    os.makedirs('match_timelines')

# Get match timelines
for match_id in selected_match_ids_list:
    # Apply rate limiting
    rl_sec()
    rl_min()

    # Get match timeline
    match_timeline = get_match_timeline(match_id)

    # Store the match timeline JSONs
    match_tl_dict[match_id] = match_timeline

    # Save the JSON data to a file inside the 'match_timeline' folder
    with open(os.path.join('match_timelines', f'{match_id}.json'), 'w') as json_file:
        json.dump(match_timeline, json_file)


# connect to postgresql

In [44]:
from sqlalchemy import text
from sqlalchemy.exc import OperationalError
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

def read_txt(filename):
    with open(filename, 'r') as file:
        return file.read().strip()

password = read_txt('postgres_pw.txt')
host = 'localhost'
database_name = 'loldb'
port = '5432'
engine = create_engine(f'postgresql://postgres:{password}@{host}:{port}/{database_name}')

# Try connecting and executing a simple query
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Successfully connected!")
except OperationalError as e:
    print(f"Connection failed: {e}")

from sqlalchemy import create_engine, MetaData

# Get the metadata of the database
metadata = MetaData()
metadata.reflect(bind=engine)

# List all table names
table_names = metadata.tables.keys()

print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Successfully connected!
Tables in the database:
MATCH_TIMELINE


# Split JSONs into tables and load to postgreSQL

In [53]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="your_host",
    database="loldb",
    user="your_username",
    password="your_password"
)

# Get the table schema information
cursor = conn.cursor()
cursor.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE'")
table_names = cursor.fetchall()

# Print the table names
for table_schema, table_name in table_names:
    print(f"Table Schema: {table_schema}, Table Name: {table_name}")

# Close the cursor and connection
cursor.close()
conn.close()

OperationalError: connection to server at "your_host" (143.244.220.150), port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [48]:
from sqlalchemy import text
from sqlalchemy.exc import OperationalError

try:
    with engine.connect() as connection:
        result = connection.execute(text('SELECT * FROM "MATCH_TIMELINE"'))
        print("Successfully connected!")

        # Fetch all records and print them
        rows = result.fetchall()

        if not rows:
            print("The table is empty. No records to fetch.")
        else:
            for row in rows:
                print(row)

except OperationalError as e:
    print(f"Connection failed: {e.orig}")

Successfully connected!
The table is empty. No records to fetch.


In [52]:
import os
import json
from sqlalchemy import create_engine, Table, Column, Integer, String, JSON, MetaData
from sqlalchemy.sql import insert

# Define the table structure
metadata = MetaData()
match_timeline_table = Table(
    'MATCH_TIMELINE',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('region', String, nullable=False),
    Column('match_id', Integer, nullable=False),
    Column('timeline_json', JSON, nullable=False)
)

# Create the table in the database (if it doesn't exist)
metadata.create_all(engine)

total_rows_added = 0

# Connect to the database
with engine.connect() as connection:

    # Loop through each file in the match_timelines directory
    for filename in os.listdir('match_timelines'):

        # Ensure the file is a JSON file
        if filename.endswith('.json'):

            # Extract the region and match_id from the filename
            region, match_id = filename.split('_') # Changed this line
            match_id = match_id.split('.')[0]

            # In case the match_id in the filename is not a valid integer,
            # ignore this file and move on to the next one
            try:
                match_id = int(match_id)
            except ValueError as e:
                print(f"ValueError occurred: {e}")
                continue

            # Open the file and load the JSON data
            with open(os.path.join('match_timelines', filename), 'r') as json_file:
                match_timeline = json.load(json_file)

            # Create an INSERT statement for the match_timeline
            stmt = insert(match_timeline_table).values(region=region, match_id=match_id, timeline_json=match_timeline)

            # Execute the statement and increment the total_rows_added by the number of rows affected
            try:
                result = connection.execute(stmt)
                rows_added = result.rowcount
                total_rows_added += rows_added
                print(f"Rows added: {rows_added}")
            except Exception as e:
                print(f"Error occurred when executing INSERT statement: {e}")

print(f"Total rows added: {total_rows_added}")

Error occurred when executing INSERT statement: (psycopg2.errors.UndefinedColumn) column "region" of relation "MATCH_TIMELINE" does not exist
LINE 1: INSERT INTO "MATCH_TIMELINE" (region, match_id, timeline_jso...
                                      ^

[SQL: INSERT INTO "MATCH_TIMELINE" (region, match_id, timeline_json) VALUES (%(region)s, %(match_id)s, %(timeline_json)s) RETURNING "MATCH_TIMELINE".id]
[parameters: {'region': 'NA1', 'match_id': 4334085229, 'timeline_json': '{"metadata": {"dataVersion": "2", "matchId": "NA1_4334085229", "participants": ["pEzq-TwsSVlh_qVqqKbSP-LcHaCSlPSHzYoksh86q-DzJ3F1-uOXY-VL_v4oKc7WsXzN ... (767365 characters truncated) ... iGxzfH7DGxAcBpuFpUDFIoDylKtvCg"}, {"participantId": 10, "puuid": "lbpYhDIEMOhsFmgtcg8JgiJ3ONencq5fiKJqxWxwE4HCpXK3jnUZV9CInwqsNzwFLE9ztahbpg4zhw"}]}}'}]
(Background on this error at: https://sqlalche.me/e/20/f405)
Error occurred when executing INSERT statement: (psycopg2.errors.InFailedSqlTransaction) current transaction is abort

In [ ]:
# import json
# league_id = '008da363-45af-4a07-87d2-f22567e27213'

# summoner_ids = get_summoner_ids_from_league_id(league_id)

# puuid = get_puuid_by_summon_id(summoner_ids[0])

# match_ids = get_match_ids_by_puuid(puuid)

# match_timeline = get_match_timeline(match_ids[0])

# timeline_events = extract_events_from_timeline(match_timeline)